In [60]:
import dateutil
import datetime
import tweepy

### Tweepy

In [6]:
# Creating the authentication object
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# Setting your access token and secret
auth.set_access_token(access_token, access_token_secret)
# Creating the API object while passing in auth information
api = tweepy.API(auth) 

In [127]:
tweets = []
last_date = datetime.datetime.now().strftime('%Y-%m-%d')
for k in range(10):
    r = api.search(q="#machinelearningflashcards", count=100, until=last_date)
    tweets += [(k._json['text'], k._json['created_at']) for k in r if k._json['user']['screen_name'] == 'chrisalbon' and 
                                                                          'media' in k._json['entities']]
    dates = [dateutil.parser.parse(k[1]).strftime('%Y-%m-%d') for k in tweets]
    last_date = min(dates)
tweets

[('Confusion Matrix #machinelearningflashcards https://t.co/GVk5CMBAQa',
  'Thu Jul 06 19:41:48 +0000 2017'),
 ('Out-Of-Bag Error #machinelearningflashcards https://t.co/oHZwQZLJtQ',
  'Thu Jul 06 17:09:00 +0000 2017'),
 ('MSE Vs MAE #machinelearningflashcards https://t.co/dn7QP5mLNt',
  'Wed Jul 05 19:10:12 +0000 2017'),
 ('Explained Sum Of Squares #machinelearningflashcards https://t.co/P4axGoRTqF',
  'Wed Jul 05 17:50:59 +0000 2017'),
 ('Non-Parametric Methods #machinelearningflashcards https://t.co/9UFyDv6c69',
  'Tue Jul 04 19:36:50 +0000 2017'),
 ('Training Error Rate #machinelearningflashcards https://t.co/RkigF5Qvb9',
  'Tue Jul 04 18:02:50 +0000 2017'),
 ('Random Forest Classification #machinelearningflashcards https://t.co/aLzGWvyAAH',
  'Mon Jul 03 19:44:17 +0000 2017'),
 ('Model Complexity #machinelearningflashcards https://t.co/V9LVOEczrh',
  'Mon Jul 03 17:59:35 +0000 2017'),
 ('Learning Curve #machinelearningflashcards https://t.co/NHk4ogDCNC',
  'Fri Jun 30 19:37:23 +00

### Selenium

In [56]:
from selenium import webdriver
import urllib
import time
import numpy as np
from bs4 import BeautifulSoup

In [52]:
driver = webdriver.Chrome()
driver.set_page_load_timeout(15)

In [53]:
query = '#machinelearningflashcards'
# driver.get("https://twitter.com/search?q={}".format(urllib.parse.quote(query)))
driver.get("https://twitter.com/search?f=tweets&vertical=default&q={}".format(urllib.parse.quote(query)))

In [54]:
length = []
while True:
    time.sleep(np.random.randint(30,100)*0.01)
    tweets_found = driver.find_elements_by_class_name('tweet') 
    driver.execute_script("return arguments[0].scrollIntoView();", tweets_found[::-1][0])
    # Stop the loop while no more found tweets 
    length.append(len(tweets_found))
    if len(length) > 3 and (len(length) - len(set(length))) > 2:
        print('No more tweets (%d) for this query' % len(tweets_found))
        break

No more tweets (352) for this query


In [109]:
tweets = []
for tweet in tweets_found:
    tweet_dict = {}
    tweet = tweet.get_attribute('innerHTML')
    bs = BeautifulSoup(tweet.strip(), "lxml")
    tweet_dict['username'] = bs.find('span', class_='username').text
    tweet_dict['date'] = [v for k, v in bs.find('a', class_='tweet-timestamp').attrs.items() if 'title' in k][0]
    tweet_dict['text'] = bs.find('p', class_='tweet-text').text
    try:
        tweet_dict['images'] = [k['src'] for k in bs.find('div', class_="AdaptiveMedia-container").find_all('img')]
    except:
        tweet_dict['images'] = []
    if len(tweet_dict['images']) > 0:
        tweet_dict['text'] = tweet_dict['text'][:tweet_dict['text'].index('pic.twitter')-1]
    tweets.append(tweet_dict)

In [135]:
# We keep only tweets by chrisalbon with pictures
search_tweets = [tw for tw in tweets if tw['username'] == '@chrisalbon' and len(tw['images']) > 0]
# He made multiple tweets on the same topic, we keep only the most recent tweets
# We use the indexes of the reversed tweet list and dictionnaries to keep only key 
unique_search_index = sorted(list({t['text'].lower():i for i,t in list(enumerate(search_tweets))[::-1]}.values()))
unique_search_tweets = [search_tweets[i] for i in unique_search_index]